In [ ]:
from cedne import cedne
from cedne import utils
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
w = utils.makeWorm()
nn = w.networks["Neutral"]
# nprc = 'NTC_1_NTR_1'
# utils.loadNeuropeptides(w, neuropeps=nprc)
utils.loadNeuropeptides(nn)

In [ ]:
# sigNeurList_corrdiff = ['ADFL',
#                         'ADFR',
#   'ASEL',
#   'AWAL',
#   'AWAR'
#   'AWCL',
#   'AWCR'
#   'ASGL',
#   'ASGR'
#   'URYDL',
#   'URYDR',
#   'ADAL',
#   'ADAR',
#   'AIML'
#   'AIMR'
#   'RIML',
#   'RIMR',
#   'RMEL', 
#   'RMER']

In [ ]:
sigNeurList_corrdiff = [
  'ASEL',
  'AWCL',
  'AWCR',
  'AIAL',
  'AIAR',
  'AIBL',
  'AIBR',
  'AIYL',
  'AIYR',
  'AIZL',
  'AIZR',
  'RIAL',
  'RIAR',
]

In [ ]:
sigNeurList_corrdiff = [n for n in nn.neurons if nn.neurons[n].category in ["head motor neuron", "sublateral motor neuron", "SN6"]]

In [ ]:
for e, c in nn.neurons['ADAL'].get_connections(direction='in').items():
    print(e, c.connection_type)

In [ ]:
len(sigNeurList_corrdiff)

In [ ]:
plotList = sigNeurList_corrdiff
nps, nps_base = [], []
num_rands = 1000
for e,c in nn.connections.items():
    if not c.connection_type  in ['chemical-synapse', 'gap-junction']:
        nps_base.append(c.connection_type)
        if e[1].name in plotList:
                nps.append(c.connection_type)

nps_random = []
for j in np.random.randint(0, len(nn.neurons), (num_rands,len(plotList))):
    rand_neuropeps = []
    rand_plotlist = [list(nn.neurons)[n] for n in j]
    for v in rand_plotlist:
        for e,c in nn.neurons[v].get_connections(direction='in').items():
            if not c.connection_type  in ['chemical-synapse', 'gap-junction']: 
                rand_neuropeps.append(c.connection_type)
    nps_random.append(rand_neuropeps)

perc_NP = {}
perc_NP_base = {}
perc_NP_random = {}

val_NP = {}
val_NP_base = {}
val_NP_random = {}
for n in set(nps):
    perc_NP[n] = len([n1 for n1 in nps if n1 == n])*100/len(nps)
    perc_NP_base[n] = len([n1 for n1 in nps_base if n1 == n])*100/len(nps_base) 
    perc_NP_random[n] = [len([n1 for n1 in nps_rand if n1 == n])*100/len(nps_rand) for nps_rand in nps_random]

    val_NP[n] = len([n1 for n1 in nps if n1 == n])
    val_NP_base[n] = len([n1 for n1 in nps_base if n1 == n])
    val_NP_random[n] = [len([n1 for n1 in nps_rand if n1 == n]) for nps_rand in nps_random]

neurpep, percval = zip(*sorted(perc_NP.items(), key=lambda x:x[1], reverse=True))
percval_base = [perc_NP_base[n] for n in neurpep]
percval_random_mu = [np.mean(perc_NP_random[n]) for n in neurpep]
percval_random_sigma = [np.std(perc_NP_random[n]) for n in neurpep]

val = [val_NP[n] for n in neurpep]  
val_base = [val_NP_base[n] for n in neurpep] 


In [ ]:
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests

In [ ]:
# zscore_sig_thres = 2.326
p_thres = 0.05

f, ax = plt.subplots(figsize=(21,3))
ax.scatter(range(len(neurpep)), percval, color='orange')
# ax.scatter(range(len(neurpep)), percval_base, color='k')
ax.errorbar(range(len(neurpep)), percval_random_mu, yerr= percval_random_sigma, color='gray', linestyle='None', marker='o')

zscores = [(percval[l]- percval_random_mu[l])/percval_random_sigma[l] for l in np.arange(len(neurpep))]
p_values_right = 1 - norm.cdf(zscores)
# _, p_bh_right, _, _ = multipletests(p_values_right, method='bonferroni')
_, p_bh_right, _, _ = multipletests(p_values_right, method='fdr_bh')

#zscore_sig = np.where(np.array([(percval[l]- percval_random_mu[l])/percval_random_sigma[l] for l in np.arange(len(neurpep))])>zscore_sig_thres)[0]
zscore_sig = np.where(p_bh_right<p_thres)[0]
for x in zscore_sig:
        ax.text(x=x, y=16, s="*", color='k', ha='center')
utils.simpleaxis(ax)
ax.set_xticks(range(len(neurpep)), labels=neurpep, rotation=45, horizontalalignment='right')

labels = ax.get_xticklabels()
ax.set_ylabel("% incoming neuropeptidergic\n connections")

ax.set_yticks([0,8,16])
# Make specific labels bold
for x in zscore_sig:
        labels[x].set_fontweight('bold')
plt.savefig("Enriched_neuropeptides.svg", transparent=True)
plt.show()

In [ ]:
insig = 0
total = 0
for e,c in nn.connections.items():
    if c.connection_type == 'FLP_9_DMSR_1':
        if e[1].name in sigNeurList_corrdiff:
            insig+=1
            print(e[0].name, e[1].name, e[2])
    total+=1

In [ ]:
insig/total

In [ ]:
for l, (p,n,p_mul) in enumerate(zip(percval, neurpep, p_bh_right)):
    print(p,n, percval_random_mu[l], (percval[l] -percval_random_mu[l])/percval_random_sigma[l], p_mul)

In [ ]:
zscores_np = []
for j,n in enumerate(neurpep):
    zscores_np.append((np.array(perc_NP_random[n]) - np.array(percval_random_mu[j]))/np.array(percval_random_sigma[j]))

In [ ]:
plt.pcolor(np.corrcoef(zscores_np), vmin=-1, vmax=1, cmap='PuOr')
plt.show()

In [ ]:
corrm = np.abs(np.corrcoef(zscores_np))
plt.hist(np.ravel(corrm[np.triu_indices_from(corrm, k=1)]), bins=30, density=True, cumulative=True, histtype='step')
plt.axvline(x=0.3)
plt.axhline(y=0.8)
plt.show()

In [ ]:
len(zscore_sig)